In [1]:
import pandas
import glob

## read in data

In [2]:
df = pandas.DataFrame()
filelist = glob.glob("C:\Users\Rich\Documents\GitHub\CitiBikeApp\data\*.csv")
  
#print filelist
    
#for afile in filelist[-12:-1]:
temp_df = pandas.read_csv(filelist[0])
try:
    #temp_df = temp_df[['starttime','start station latitude','start station longitude']].sample(frac=0.50)
    temp_df = temp_df[['starttime','start station name','end station name']]
    df = pandas.concat([df, temp_df], ignore_index=True)
except KeyError:
    print filelist[0]

## convert datetimes and get some relevant time information in table columns

In [3]:
#df['starttime'] = pandas.to_datetime(df['starttime'],format='%Y-%m-%d %H:%M:%S')
#df['Datemod'] = df['starttime'].apply(lambda x: str(x.year)+str(x.month)+str(x.day))
from datetime import datetime
try:
    df['starttime'] = pandas.to_datetime(df['starttime'],format='%Y-%m-%d %H:%M:%S')
except ValueError:
    try:
        df['starttime'] = pandas.to_datetime(df['starttime'],format='%m/%d/%Y %H:%M:%S')
    except ValueError:
        df['starttime'] = pandas.to_datetime(df['starttime'])
        #df['starttime'] = pandas.to_datetime(df['starttime'],format='%m/%d/%Y %H:%M')
df['Year'] = df['starttime'].apply(lambda x: x.year)
df['Month'] = df['starttime'].apply(lambda x: x.month)
df['Day'] = df['starttime'].apply(lambda x: x.day)

In [4]:
df.head()

,starttime,start station name,end station name,Year,Month,Day
0,2013-07-01 00:00:00,E 47 St & 2 Ave,1 Ave & E 15 St,2013,7,1
1,2013-07-01 00:00:02,W 26 St & 10 Ave,W 20 St & 11 Ave,2013,7,1
2,2013-07-01 00:01:04,Lafayette St & E 8 St,E 11 St & 2 Ave,2013,7,1
3,2013-07-01 00:01:06,Forsyth St & Broome St,Broadway & W 60 St,2013,7,1
4,2013-07-01 00:01:10,University Pl & E 14 St,Suffolk St & Stanton St,2013,7,1


## get weather information

In [14]:
#weather data
prcp_df = pandas.read_csv("C:\Users\Rich\Documents\GitHub\CitiBikeApp\data\weather_data_noaa.csv",dtype=str)
prcp_df.head()

,STATION,STATION_NAME,DATE,PRCP,SNWD,SNOW,TMAX,TMIN,TOBS,AWND,WT09,WT01,WT06,WT05,WT02,WT11,WT04,WT08,WT03,WT10
0,GHCND:US1NYWC0003,WHITE PLAINS 3.1 NNW NY US,20130701,0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
1,GHCND:US1NYWC0003,WHITE PLAINS 3.1 NNW NY US,20130702,655,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
2,GHCND:US1NYWC0003,WHITE PLAINS 3.1 NNW NY US,20130703,0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
3,GHCND:US1NYWC0003,WHITE PLAINS 3.1 NNW NY US,20130716,0,-9999,0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
4,GHCND:US1NYWC0003,WHITE PLAINS 3.1 NNW NY US,20130717,0,-9999,0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999


In [15]:
prcp_df = prcp_df[prcp_df['STATION_NAME']=='NEW YORK CENTRAL PARK OBS BELVEDERE TOWER NY US'][['DATE','PRCP','SNOW']].reset_index()
prcp_df['Year'] = prcp_df['DATE'].apply(lambda x: int(x[0:4]))
prcp_df['Month'] = prcp_df['DATE'].apply(lambda x: int(x[4:6]))
prcp_df['Day'] = prcp_df['DATE'].apply(lambda x: int(x[6:8]))
prcp_df = prcp_df.drop(['index','DATE'],axis=1)
prcp_df.head()

,PRCP,SNOW,Year,Month,Day
0,213,0,2013,7,1
1,20,0,2013,7,2
2,135,0,2013,7,3
3,0,0,2013,7,4
4,0,0,2013,7,5


In [16]:
weather_df = pandas.read_csv("C:\Users\Rich\Documents\GitHub\CitiBikeApp\data\jfk_temp_data.txt",dtype=str)

# convert date to datetime
weather_df['Date'] = weather_df['Date']+' '+weather_df['HrMn']
weather_df['Date'] = pandas.to_datetime(weather_df['Date'], format='%Y%m%d %H%M')
weather_df = weather_df.drop('HrMn',1)
weather_df = weather_df.drop('QCP',1)
weather_df = weather_df.drop('USAF',1)
weather_df = weather_df.drop('NCDC.',1)
weather_df = weather_df.drop('I',1)
weather_df = weather_df.drop('Type',1)
#weather_df = weather_df.drop('Q',1)
weather_df['Year'] = weather_df['Date'].apply(lambda x: x.year)
weather_df['Month'] = weather_df['Date'].apply(lambda x: x.month)
weather_df['Day'] = weather_df['Date'].apply(lambda x: x.day)
weather_df['Hour'] = weather_df['Date'].apply(lambda x: x.hour)
weather_df['Temp'] = weather_df['Temp'].apply(lambda x: float(x))
weather_df.head()

,Date,Temp,Q,Year,Month,Day,Hour
0,2013-07-01 00:00:00,23.3,1,2013,7,1,0
1,2013-07-01 00:49:00,23.0,5,2013,7,1,0
2,2013-07-01 00:51:00,22.8,5,2013,7,1,0
3,2013-07-01 01:05:00,23.0,5,2013,7,1,1
4,2013-07-01 01:51:00,22.2,5,2013,7,1,1


In [17]:
test_weather_df = pandas.DataFrame({'avg_temp': weather_df['Temp'].groupby([weather_df['Year'],weather_df['Month'],weather_df['Day']]).mean()}).reset_index()
test_weather_df.head()

,Year,Month,Day,avg_temp
0,2013,7,1,22.808475
1,2013,7,2,22.888000
2,2013,7,3,23.530556
3,2013,7,4,25.092500
4,2013,7,5,25.981250


In [18]:
test_weather_df = pandas.merge(prcp_df,test_weather_df,on=['Year','Month','Day'])
test_weather_df.head()

,PRCP,SNOW,Year,Month,Day,avg_temp
0,213,0,2013,7,1,22.808475
1,20,0,2013,7,2,22.888000
2,135,0,2013,7,3,23.530556
3,0,0,2013,7,4,25.092500
4,0,0,2013,7,5,25.981250


In [19]:
result = pandas.merge(df,test_weather_df,on=['Year','Month','Day'])
result.head()

,starttime,start station name,end station name,Year,Month,Day,PRCP,SNOW,avg_temp
0,2013-07-01 00:00:00,E 47 St & 2 Ave,1 Ave & E 15 St,2013,7,1,213,0,22.808475
1,2013-07-01 00:00:02,W 26 St & 10 Ave,W 20 St & 11 Ave,2013,7,1,213,0,22.808475
2,2013-07-01 00:01:04,Lafayette St & E 8 St,E 11 St & 2 Ave,2013,7,1,213,0,22.808475
3,2013-07-01 00:01:06,Forsyth St & Broome St,Broadway & W 60 St,2013,7,1,213,0,22.808475
4,2013-07-01 00:01:10,University Pl & E 14 St,Suffolk St & Stanton St,2013,7,1,213,0,22.808475


In [21]:
# add in a new column for weekday
result['Month'] = result['Month'].apply('{:0>2}'.format)
result['Day'] = result['Day'].apply('{:0>2}'.format)
result['weekday'] = result['Year'].map(str)+'-'+result['Month']+'-'+result['Day']
result['weekday'] = pandas.to_datetime(result['weekday'], format='%Y-%m-%d')
result['weekday'] = result['weekday'].apply(lambda x: x.weekday())
#result['neighborhood'] = result['neighborhood'].map(str)
#result['avg_temp'] = result['avg_temp'].apply(lambda x: int(round(x)))
result['avg_temp'] = result['avg_temp'].apply(lambda x: float(x))
result['weekday'] = result['weekday'].apply(lambda x: str(x))
result['PRCP'] = result['PRCP'].map(int)
result['SNOW'] = result['SNOW'].map(int)
result.head()

,starttime,start station name,end station name,Year,Month,Day,PRCP,SNOW,avg_temp,weekday
0,2013-07-01 00:00:00,E 47 St & 2 Ave,1 Ave & E 15 St,2013,07,01,213,0,22.808475,0
1,2013-07-01 00:00:02,W 26 St & 10 Ave,W 20 St & 11 Ave,2013,07,01,213,0,22.808475,0
2,2013-07-01 00:01:04,Lafayette St & E 8 St,E 11 St & 2 Ave,2013,07,01,213,0,22.808475,0
3,2013-07-01 00:01:06,Forsyth St & Broome St,Broadway & W 60 St,2013,07,01,213,0,22.808475,0
4,2013-07-01 00:01:10,University Pl & E 14 St,Suffolk St & Stanton St,2013,07,01,213,0,22.808475,0


## model

In [22]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.feature_extraction import DictVectorizer as DV

#X = result.as_matrix(columns=['neighborhood', 'avg_temp', 'weekday'])
#y = result['count'].values
X = result[['start station name', 'avg_temp', 'weekday','PRCP','SNOW']]
y = result['end station name']

#one-hot encoding
#enc = OneHotEncoder(categorical_features='all', sparse=True)
vectorizer = DV(sparse=False)
X = X[['start station name','avg_temp','weekday','PRCP','SNOW']].T.to_dict().values()
X = vectorizer.fit_transform(X)
y = y.values

#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [31]:
print X_train.shape
print len(y)

(590391L, 339L)
843416


In [ ]:
from sklearn.cross_validation import cross_val_score, train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV

## set the parameters by cross-validation                                                                                                                          
cv = KFold(n=len(X_train), n_folds=10, shuffle=True)
param_grid = {'n_neighbors':[5, 10, 20, 30, 40, 50, 60, 70, 80, 90]}
grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=cv)
gridfit = grid.fit(X_train, y_train)
#bestparam = int(grid.best_params_['n_neighbors'])
#print bestparam

#fitter = KNeighborsRegressor(n_neighbors=bestparam)
#fitter.fit(X_t, y)

## try leaving out the weather info

In [8]:
# add in a new column for weekday
result = df
result['Month'] = result['Month'].apply('{:0>2}'.format)
result['Day'] = result['Day'].apply('{:0>2}'.format)
result['hour'] = df['starttime'].apply(lambda x: x.hour)
result['weekday'] = result['Year'].map(str)+'-'+result['Month']+'-'+result['Day']
result['weekday'] = pandas.to_datetime(result['weekday'], format='%Y-%m-%d')
result['weekday'] = result['weekday'].apply(lambda x: str(x.weekday()))

In [9]:
result.head()

,starttime,start station name,end station name,Year,Month,Day,weekday,hour
0,2013-07-01 00:00:00,E 47 St & 2 Ave,1 Ave & E 15 St,2013,07,01,0,0
1,2013-07-01 00:00:02,W 26 St & 10 Ave,W 20 St & 11 Ave,2013,07,01,0,0
2,2013-07-01 00:01:04,Lafayette St & E 8 St,E 11 St & 2 Ave,2013,07,01,0,0
3,2013-07-01 00:01:06,Forsyth St & Broome St,Broadway & W 60 St,2013,07,01,0,0
4,2013-07-01 00:01:10,University Pl & E 14 St,Suffolk St & Stanton St,2013,07,01,0,0


In [10]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.feature_extraction import DictVectorizer as DV

#X = result.as_matrix(columns=['neighborhood', 'avg_temp', 'weekday'])
#y = result['count'].values
X = result[['start station name', 'hour', 'weekday']]
y = result['end station name']

#one-hot encoding
#enc = OneHotEncoder(categorical_features='all', sparse=True)
vectorizer = DV(sparse=True)
X = X[['start station name','hour', 'weekday']].T.to_dict().values()
X = vectorizer.fit_transform(X)
y = y.values

#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
from sklearn.cross_validation import cross_val_score, train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV

## set the parameters by cross-validation                                                                                                                          
#cv = KFold(n=len(y_train), n_folds=3, shuffle=True)
#param_grid = {'n_neighbors':[5, 10, 25]}
#grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=cv)
#gridfit = grid.fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=10, p=2, weights='uniform')

In [25]:
print knn.predict(X_test[4])[0]
print y_test[4]

2 Ave & E 31 St
Barclay St & Church St


In [26]:
knn.score(X_test[0:100], y_test[0:100])

0.059999999999999998